<a href="https://colab.research.google.com/github/NLP-trade-performance-predictor/QQQ-stock-prediction/blob/master/Stocks_trend_prediction_by_headlines_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init Notebook


*   mount drive
*   install `transformers`
*   import packages
*   import `BERT`



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qq transformers

     |████████████████████████████████| 2.3MB 9.5MB/s 
     |████████████████████████████████| 901kB 36.5MB/s 
     |████████████████████████████████| 3.3MB 37.0MB/s 


In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, BertModel, AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import plotly.express as px

import numpy as np
import pandas as pd
import datetime as dt
import pprint
from collections import Counter 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
FIN_PRE_TRAINED_MODEL_NAME = "ProsusAI/finbert"

In [ ]:
# tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
fin_tokenizer = AutoTokenizer.from_pretrained(FIN_PRE_TRAINED_MODEL_NAME)

In [ ]:
# model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=3)
finbert = AutoModelForSequenceClassification.from_pretrained(FIN_PRE_TRAINED_MODEL_NAME)

# model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
# model = model.to(device=device)
finbert = finbert.to(device=device)

In [ ]:
labels_tr = { 0: 2,
              1: 0,
              2: 1 }

# Helpers

In [ ]:
PRINT_EVERY = 20
def train_epoch(model, data_loader, loss_fn, optimizer):
  losses = []
  correct_predictions = 0
  
  for i, data in enumerate(data_loader):
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)

    labels = data['labels'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask).logits
    # y_pred_softmax = torch.log_softmax(outputs, dim=1)
    y_pred_softmax = outputs
    _, predicted_labels = torch.max(y_pred_softmax, dim=1)

    predicted_labels = [labels_tr[x.item()] for x in predicted_labels]
    
    correct_predictions += len([i for i, j in zip(predicted_labels, labels.tolist()) if i == j])
    # correct_predictions += torch.sum(predicted_labels == labels)

    # calculate and save loss
    loss = loss_fn(outputs, labels)
    losses.append(loss.item())
    if i % PRINT_EVERY == 0:
      print(np.mean(losses))

    # optimizer step
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  # return {'accuracy': correct_predictions.item() / len(data_loader.dataset),
  #         'loss': np.mean(losses)}

  return {'accuracy': correct_predictions / len(data_loader.dataset),
          'loss': np.mean(losses)}

In [ ]:
def comp_train_epoch(model, data_loader, loss_fn, optimizer):
  losses = []
  correct_predictions = 0
  
  for i, data in enumerate(data_loader):
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    company_input_ids = data['company_input_ids'].to(device)
    fin_input_ids = data['fin_input_ids'].to(device)
    fin_attention_mask = data['fin_attention_mask'].to(device)

    labels = data['labels'].to(device)

    outputs = model(input_ids, attention_mask, company_input_ids, fin_input_ids, fin_attention_mask).squeeze()
    predicted_labels = (outputs > 0.5).float() * 1

    # _, predicted_labels = torch.max(outputs, dim=1)
    # predicted_labels = [labels_tr[x.item()] for x in predicted_labels]
    
    # correct_predictions += len([i for i, j in zip(predicted_labels, labels.tolist()) if i == j])
    correct_predictions += torch.sum(predicted_labels == labels)

    # calculate and save loss
    loss = loss_fn(outputs, labels.float())
    losses.append(loss.item())
    if i % PRINT_EVERY == 0:
      print(np.mean(losses))
      print(outputs)
      print(predicted_labels)

    # optimizer step
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  return {'accuracy': correct_predictions.item() / len(data_loader.dataset),
          'loss': np.mean(losses)}

  # return {'accuracy': correct_predictions / len(data_loader.dataset),
  #         'loss': np.mean(losses)}

In [ ]:
def comp_eval_model(model, loss_fn, sets=['validation']):
  sets_metrics = dict()

  model.eval()
  
  with torch.no_grad():
    for set_name, dataloader in data_loaders.items():
      if set_name in sets:
        losses = []
        p_labels = []
        correct_predictions = 0
  
        for data in dataloader:
          input_ids = data['input_ids'].to(device)
          attention_mask = data['attention_mask'].to(device)
          company_input_ids = data['company_input_ids'].to(device)
          fin_input_ids = data['fin_input_ids'].to(device)
          fin_attention_mask = data['fin_attention_mask'].to(device)

          labels = data['labels'].to(device)

          outputs = model(input_ids, attention_mask, company_input_ids, fin_input_ids, fin_attention_mask).squeeze()
          # _, predicted_labels = torch.max(outputs, dim=1)
          predicted_labels = (outputs > 0.5).float() * 1

          # predicted_labels = [labels_tr[x.item()] for x in predicted_labels]
          
          # correct_predictions += len([i for i, j in zip(predicted_labels, labels.tolist()) if i == j])
          correct_predictions += torch.sum(predicted_labels == labels)

          p_labels.extend(predicted_labels)

          # calculate and save loss
          losses.append(loss_fn(outputs, labels.float()).item())

        sets_metrics[set_name] = {'accuracy': correct_predictions.item() / len(dataloader.dataset),
                                  'loss': np.mean(losses),
                                  'pred': [ x.item() for x in p_labels]}

        # sets_metrics[set_name] = {'accuracy': correct_predictions / len(dataloader.dataset),
        #                           'loss': np.mean(losses),
        #                           'pred': p_labels}

  model.train()

  return sets_metrics


In [ ]:
def eval_model(model, loss_fn, sets=['validation']):
  sets_metrics = dict()

  model.eval()
  
  with torch.no_grad():
    for set_name, dataloader in data_loaders.items():
      if set_name in sets:
        losses = []
        p_labels = []
        correct_predictions = 0
  
        for data in dataloader:
          input_ids = data['input_ids'].to(device)
          attention_mask = data['attention_mask'].to(device)

          labels = data['labels'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask).logits
          # y_pred_softmax = torch.log_softmax(outputs, dim=1)
          y_pred_softmax = outputs
          _, predicted_labels = torch.max(y_pred_softmax, dim=1)

          predicted_labels = [labels_tr[x.item()] for x in predicted_labels]

          p_labels.extend(predicted_labels)

          correct_predictions += len([i for i, j in zip(predicted_labels, labels.tolist()) if i == j])
          # correct_predictions += torch.sum(predicted_labels == labels)

          # calculate and save loss
          losses.append(loss_fn(outputs, labels).item())

        # sets_metrics[set_name] = {'accuracy': correct_predictions.item() / len(dataloader.dataset),
        #                           'loss': np.mean(losses),
        #                           'pred': [ x.item() for x in p_labels]}

        sets_metrics[set_name] = {'accuracy': correct_predictions / len(dataloader.dataset),
                                  'loss': np.mean(losses),
                                  'pred': p_labels}

  model.train()

  return sets_metrics

In [ ]:
def train(model, n_epochs, train_data_loader, loss_fn, optimizer):
  history = []

  for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/{n_epochs} Metrics')
    
    current_metrics = { 'train': 
      comp_train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer
      ) 
    }
    current_metrics.update(comp_eval_model(model, loss_fn))

    pprint.pprint(current_metrics, indent=4)
    print('-' * 10)

    history.append(current_metrics)

  return history

In [ ]:
def plot(history):
  metrics_map = dict()

  for e_sets in history:
    for set_name, set_metrics in e_sets.items():
      for metric_name, metric_value in set_metrics.items():

        if metric_name not in metrics_map:
          metrics_map[metric_name] = dict()
        if set_name not in metrics_map[metric_name]:
          metrics_map[metric_name][set_name] = []

        metrics_map[metric_name][set_name].append(metric_value)
      
  for metric_name, sets in metrics_map.items():
    df = None
    for set_name, set_metrics in sets.items():
      size = len(set_metrics)
      if df is None:
        df = pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size})
      else:
        df = df.append(pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size}), ignore_index=True)

    fig = px.line(df, x="epoch", y=metric_name, line_group="set", title=f"epoch {metric_name} per dataset", color="set", hover_name="set")
    fig.show()

In [ ]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    return acc

def binary_accuracy(preds, y):
    rounded_preds = torch.round(preds)

    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

# Import Dataset



*   import `articles` and `stocks` csv files
*   join them into one `pandas` dataframe by `date`



In [ ]:
PATH='/content/drive/MyDrive/NLP Final Project'

In [ ]:
articles = pd.read_csv(f'{PATH}/articles.csv', header=None)

In [ ]:
articles.head(10)

,0,1,2
0,Facebook and Google reveal plans to build subs...,Mon 29 Mar 2021 12:34:00 GMT,CNBC
1,What Does a Google Partnership Mean for Wix? -...,Sun 28 Mar 2021 13:00:00 GMT,The Motley Fool
2,Classic Hangouts to Google Chat transition: Wh...,Mon 29 Mar 2021 14:00:00 GMT,TechRepublic
3,Google: No Benefit to An Artificially Flat URL...,Sun 28 Mar 2021 18:25:50 GMT,Search Engine Journal
4,Mtn. View signs off on Google's massive plan f...,Sun 28 Mar 2021 03:50:14 GMT,Los Altos Town Crier
5,Google Aims to Be the Anti-Amazon of E-Commerc...,Mon 29 Mar 2021 01:42:00 GMT,The New York Times
6,Google's App Store Privacy Labels: What They M...,Sun 28 Mar 2021 04:22:26 GMT,Gadgets 360
7,Ex-Google Policy Lead Founds Tech-Friendly Pro...,Mon 29 Mar 2021 12:04:28 GMT,Yahoo Finance
8,Google Facebook Twitter grilled in US on fake ...,Thu 25 Mar 2021 20:07:06 GMT,BBC News
9,Google offered a professor $60000 but he turne...,Wed 24 Mar 2021 21:46:00 GMT,CNN


In [ ]:
articles['companies'] = pd.Series([[]] * len(articles))

In [ ]:
COMPANIES = {"apple": "AAPL",
             "facebook": "FB",
             "google": "GOOG",
             "microsoft": "MSFT",
             "tesla": "TSLA",
             "amazon": "AMZN",
             "paypal": "PYPL",
             "intel": "INTC",
             "nvidia": "NVDA",
             "alphabet": "GOOGL"}

In [ ]:
articles['companies'] = articles[0].map(lambda headline: [comp for comp in COMPANIES.keys() if comp in headline.lower()])

In [ ]:
articles = articles[articles['companies'].map(len) > 0]

In [ ]:
comps = list()

for i, row in articles.iterrows():
    comps.extend(row['companies'])

set(comps)

{'alphabet',
 'amazon',
 'apple',
 'facebook',
 'google',
 'intel',
 'microsoft',
 'nvidia',
 'paypal',
 'tesla'}

In [ ]:
articles = articles.explode('companies', ignore_index=False)

In [ ]:
articles['stock'] = articles['companies'].map(lambda comp: COMPANIES[comp])

In [ ]:
articles[1] = articles[1].map(lambda article: dt.datetime.strftime(dt.datetime.strptime(article, '%a %d %b %Y %H:%M:%S GMT'), '%Y-%m-%d'))

In [ ]:
articles

,0,1,2,companies,stock
0,Facebook and Google reveal plans to build subs...,2021-03-29,CNBC,facebook,FB
0,Facebook and Google reveal plans to build subs...,2021-03-29,CNBC,google,GOOG
1,What Does a Google Partnership Mean for Wix? -...,2021-03-28,The Motley Fool,google,GOOG
2,Classic Hangouts to Google Chat transition: Wh...,2021-03-29,TechRepublic,google,GOOG
3,Google: No Benefit to An Artificially Flat URL...,2021-03-28,Search Engine Journal,google,GOOG
...,...,...,...,...,...
4595,Google Owns a Satellite Now - The Atlantic,2014-06-10,The Atlantic,google,GOOG
4596,CNN – How Google's Calico aims to fight aging ...,2013-10-03,CNN,google,GOOG
4597,Google Employees' Favorite Perks - Business In...,2014-07-02,Business Insider,google,GOOG
4598,An Excellent Explanation Of Why Google Works L...,2013-07-10,Forbes,google,GOOG


In [ ]:
stocks = pd.read_csv(f'{PATH}/stocks_unified.csv', header=None)

In [ ]:
stocks

,0,1,2,3,4
0,2014-09-18,-0.1400,-0.001373,1,AAPL
1,2014-05-12,5.3400,0.009090,2,AAPL
2,2013-01-11,-0.7000,-0.001344,1,AAPL
3,2011-12-16,0.6600,0.001735,1,AAPL
4,2010-06-29,-7.9505,-0.030102,0,AAPL
...,...,...,...,...,...
39409,2017-03-31,-0.4300,-0.001543,1,TSLA
39410,2014-02-10,7.2200,0.038132,2,TSLA
39411,2012-02-09,0.5800,0.018125,2,TSLA
39412,2011-09-26,-1.0000,-0.037707,0,TSLA


In [ ]:
joined_df = articles.set_index([1, 'stock']).join(stocks.set_index([0, 4]), lsuffix='article', rsuffix='stock', on=[1, 'stock'])
joined_df = joined_df.dropna()
joined_df = joined_df.rename({0: 'headline', '2article': 'publisher', 1: 'diff', '2stock': 'ratio', 3: 'label', 'companies': 'company'}, axis=1)
joined_df['headline'] = joined_df['headline'].map(lambda headline: headline[:headline.rindex(' - ')])
joined_df['label'] = joined_df['label'].astype(int)
joined_df = joined_df.reset_index()
joined_df = joined_df.rename({1: 'date'}, axis=1)
joined_df

,date,stock,headline,publisher,company,diff,ratio,label
0,2021-03-29,FB,Facebook and Google reveal plans to build subs...,CNBC,facebook,5.0500,0.017672,2
1,2021-03-29,GOOG,Facebook and Google reveal plans to build subs...,CNBC,google,28.0700,0.013842,2
2,2021-03-29,GOOG,Classic Hangouts to Google Chat transition: Wh...,TechRepublic,google,28.0700,0.013842,2
3,2021-03-29,GOOG,Google Aims to Be the Anti-Amazon of E-Commerc...,The New York Times,google,28.0700,0.013842,2
4,2021-03-29,AMZN,Google Aims to Be the Anti-Amazon of E-Commerc...,The New York Times,amazon,20.2906,0.006641,2
...,...,...,...,...,...,...,...,...
3908,2014-05-12,GOOG,Research in India suggests Google search resul...,Washington Post,google,6.4100,0.012244,2
3909,2013-11-15,FB,Google reportedly tried to outbid Facebook for...,The Verge,facebook,-0.1000,-0.002036,1
3910,2014-06-10,GOOG,Google Owns a Satellite Now,The Atlantic,google,0.0400,0.000071,1
3911,2014-07-02,GOOG,Google Employees' Favorite Perks,Business Insider,google,-1.0150,-0.001740,1


In [ ]:
# joined_df["headline"] = joined_df["company"] + pd.Series(["[SEP]"]*len(joined_df)) + joined_df["headline"] 

In [ ]:
joined_df = joined_df.drop(['stock', 'publisher', 'diff', 'ratio'], axis=1)

In [ ]:
joined_df.head(10)

,date,headline,company,label
0,2021-03-29,Facebook and Google reveal plans to build subs...,facebook,2
1,2021-03-29,Facebook and Google reveal plans to build subs...,google,2
2,2021-03-29,Classic Hangouts to Google Chat transition: Wh...,google,2
3,2021-03-29,Google Aims to Be the Anti-Amazon of E-Commerc...,google,2
4,2021-03-29,Google Aims to Be the Anti-Amazon of E-Commerc...,amazon,2
5,2021-03-29,Ex-Google Policy Lead Founds Tech-Friendly Pro...,google,2
6,2021-03-25,Google Facebook Twitter grilled in US on fake ...,facebook,0
7,2021-03-25,Google Facebook Twitter grilled in US on fake ...,google,1
8,2021-03-24,Google offered a professor $60000 but he turne...,google,0
9,2021-03-25,Google is testing Memory an upgrade for Assist...,google,1


In [ ]:
finbert.eval()
id2label = {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  }

def get_sentiment(headline):
  encoding = fin_tokenizer(headline, truncation=True, padding='max_length', max_length=64)
  input_ids = torch.tensor(encoding['input_ids']).to(device).unsqueeze(dim=0)
  attention_mask = torch.tensor(encoding['attention_mask']).to(device).unsqueeze(dim=0)

  out = finbert(input_ids, attention_mask=attention_mask).logits
  _, label = torch.max(out, dim=1)

  return id2label[str(label.item())]

get_sentiment("apple stock is sweeping as covid 19 cases rise")

'negative'

In [ ]:
joined_df['sentiment'] = joined_df['headline'].apply(get_sentiment)

In [ ]:
joined_df.groupby(['sentiment', 'label']).size()

sentiment  label
negative   0        264
           1        231
           2        269
neutral    0        916
           1        770
           2        969
positive   0        165
           1        134
           2        195
dtype: int64

In [ ]:
daily_df = (joined_df.groupby(['date'])
      .agg({'headline': lambda x: x.tolist(), 'label': lambda x: x.tolist(), 'company': lambda x: x.tolist()})
      .rename({'headline' : 'headlines','label' : 'labels', 'company': 'companies'}, axis=1)
      .reset_index())

daily_df

,date,headlines,labels,companies
0,2010-01-04,"[Report: Apple scheduling Jan. 27 media event,...","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[apple, apple, apple, apple, apple, apple, app..."
1,2010-01-05,[Microsoft offers many versions of Office 2010...,"[1, 1, 1, 1, 1, 1, 1]","[microsoft, microsoft, microsoft, microsoft, a..."
2,2010-01-06,"[Microsoft Tablet to Beat Apple to the Punch?,...","[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]","[apple, microsoft, microsoft, microsoft, micro..."
3,2010-01-07,[Binary Body Double: Microsoft Reveals the Sci...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[microsoft, microsoft, apple, apple, apple, ap..."
4,2010-01-08,[Yahoo To Keep BOSS In Microsoft Deal May Char...,"[2, 2, 2, 2, 2, 2]","[microsoft, microsoft, apple, apple, apple, ap..."
...,...,...,...,...
1321,2021-03-26,[What Are Google Web Stories? A Guide for Mark...,"[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, ...","[google, google, google, google, google, googl..."
1322,2021-03-29,[Facebook and Google reveal plans to build sub...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, ...","[facebook, google, google, google, amazon, goo..."
1323,2021-03-30,[Building a more inclusive skills-based econom...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[microsoft, microsoft, microsoft, microsoft, m..."
1324,2021-03-31,[Microsoft Touts Dev Partners in Project Reuni...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[microsoft, microsoft, microsoft, microsoft, m..."


# FinBERT - sentiment to trend

In [ ]:
daily_headlines = daily_df['headlines'].tolist()
daily_companies = daily_df['companies'].tolist()
daily_labels = daily_df['labels'].tolist()

In [ ]:
headlines_fin_encodings = [fin_tokenizer(day_headlines, truncation=True, padding='max_length', max_length=64) for day_headlines in daily_headlines]

In [ ]:
class CompanyHeadlinesStocksDataset(torch.utils.data.Dataset):
    def __init__(self, headlines_fin_encodings, companies, labels):
        self.headlines_fin_encodings = headlines_fin_encodings
        self.companies = companies
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.headlines_fin_encodings[idx].items()}
        item['companies'] = self.companies[idx]
        item['labels'] = self.labels[idx]

        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
fin_dataset = CompanyHeadlinesStocksDataset(headlines_fin_encodings, daily_companies, daily_labels)
dataloader = DataLoader(
    fin_dataset,
    batch_size=1,
    shuffle=False
  )

In [ ]:
def get_companies_gold_trend(companies, labels):
  company_labels_count = { company: Counter() for company in set(companies)}
  for company, label in zip(companies, labels):
    company_labels_count[company][label] += 1

  return {company: company_labels_count[company].most_common(1)[0][0] for company in set(companies)}


def get_companies_predicted_trend(companies, headlines_label_probs):
  company_labels_entropy = { company: 0 for company in set(companies)}
  for company, label_probs in zip(companies, headlines_label_probs):
    # (label + 1) for labels {1, 2, 3} = {down, same, up} and entropy at (1, 3)
    company_labels_entropy[company] += sum([((labels_tr[label] + 1) * prop) for label, prop in enumerate(label_probs)])

  return {company: (round(company_labels_entropy[company] / companies.count(company)) - 1) for company in set(companies)}


def accuracy(gold, pred):
  return float(len([gold[company] for company in gold.keys() if (gold[company] == pred[company])])) / len(gold.keys())
  

In [ ]:
finbert.eval()

acc = 0.0
comps = []
comp_count = []
preds = []
golds = []

with torch.no_grad():
  for data in dataloader:
    # input
    input_ids = data['input_ids'].to(device).squeeze(dim=0)
    attention_mask = data['attention_mask'].to(device).squeeze(dim=0)

    # predict sentiment for each headline
    outputs = torch.softmax(finbert(input_ids, attention_mask=attention_mask).logits, dim=1).tolist()

    # compute predicted labels and gold labels for each company
    companies = [x[0] for x in data['companies']]
    labels = [x[0].item() for x in data['labels']]

    companies_predicted_label = get_companies_predicted_trend(companies, outputs)
    companies_gold_label = get_companies_gold_trend(companies, labels)

    for comp in companies_predicted_label.keys():
      comps.append(comp)
      comp_count.append(companies.count(comp))
      preds.append(companies_predicted_label[comp])
      golds.append(companies_gold_label[comp])
    
    day_acc = accuracy(companies_gold_label, companies_predicted_label)
    # print("daily accuracy: %f" % day_acc)
    acc += day_acc

acc /= len(dataloader.dataset)
print("----------\n\ntotal accuracy: %f" % acc)

----------

total accuracy: 0.310430


In [ ]:
confusion_matrix(preds, golds)

array([[127, 120, 122],
       [683, 534, 700],
       [ 76,  55,  86]])

In [ ]:
sentiment_trend_df = pd.DataFrame({ 'company': comps,
                                    'headlines_count': comp_count,
                                    'predicted_trend': preds,
                                    'gold_trend': golds
                                    })

In [ ]:
sentiment_trend_df['company_code'] = sentiment_trend_df['company'].astype('category').cat.codes

In [ ]:
sentiment_trend_df

,company,headlines_count,predicted_trend,gold_trend,company_code
0,intel,1,1,1,5
1,apple,8,1,1,2
2,apple,3,1,1,2
3,microsoft,4,1,1,6
4,apple,6,1,0,2
...,...,...,...,...,...
2498,nvidia,11,1,2,7
2499,intel,13,1,1,5
2500,google,5,1,2,4
2501,paypal,9,1,1,8


In [ ]:
X = sentiment_trend_df[['company_code', 'predicted_trend']]
y = sentiment_trend_df['gold_trend']

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [ ]:
gnb = GaussianNB()
perceptron = Perceptron(tol=1e-3, random_state=0)
gboost = GradientBoostingClassifier(n_estimators=100, learning_rate=5e-4, max_depth=1, random_state=0)
rf = RandomForestClassifier(max_depth=1, random_state=0)

In [ ]:
classifiers = { 'gnb': gnb, 'perceptron': perceptron, 'gboost': gboost, 'random forest': rf}

In [ ]:
for name, clf in classifiers.items():
  clf.fit(X_train, y_train)
  print("%s model accuracy: %f" % (name, clf.score(X_test, y_test)))

gnb model accuracy: 0.334331
perceptron model accuracy: 0.387226
gboost model accuracy: 0.369261
random forest model accuracy: 0.377246


# Split Dataset and create data loaders

In [ ]:
df_train, df_test = train_test_split(
  joined_df,
  test_size=0.2,
)
df_train, df_val = train_test_split(
  df_train,
  test_size=0.15,
)

In [ ]:
df_train

,headline,label,company
744,Microsoft's quest to go 'carbon negative' insp...,2,microsoft
152,Google Explores Alternative to Apple's New Ant...,1,google
1530,Debate: Is Amazon The Reader's Friend?,2,amazon
1982,Tesla's Business Model Highlights What The Shi...,0,tesla
1455,Amazon files lawsuit over 'widespread tech sup...,2,amazon
...,...,...,...
524,The message Microsoft is sending to managers a...,0,microsoft
1332,Amazon argues it’s not liable for product that...,1,amazon
3028,Intel's Westmere-EX Chip to Include 10 Cores,0,intel
285,Google’s BeyondCorp Enterprise security platfo...,2,google


In [ ]:
train_encodings = tokenizer(df_train['headline'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(df_val['headline'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(df_test['headline'].tolist(), truncation=True, padding=True)

fin_train_encodings = fin_tokenizer(df_train['headline'].tolist(), truncation=True, padding=True)
fin_val_encodings = fin_tokenizer(df_val['headline'].tolist(), truncation=True, padding=True)
fin_test_encodings = fin_tokenizer(df_test['headline'].tolist(), truncation=True, padding=True)

In [ ]:
class HeadlinesStocksDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_company_encodings = tokenizer(df_train['company'].tolist(), truncation=True, padding=True)
val_company_encodings = tokenizer(df_val['company'].tolist(), truncation=True, padding=True)
test_company_encodings = tokenizer(df_test['company'].tolist(), truncation=True, padding=True)

In [ ]:
class CompanyHeadlinesStocksDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, companies_encodings, fin_encodings):
        self.encodings = encodings
        self.labels = labels
        self.companies_encodings = companies_encodings
        self.fin_encodings = fin_encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item.update({f'company_{key}': torch.tensor(val[idx]) for key, val in self.companies_encodings.items()})
        item.update({f'fin_{key}': torch.tensor(val[idx]) for key, val in self.fin_encodings.items()})

        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# train_dataset = HeadlinesStocksDataset(train_encodings, df_train['label'].tolist())
# val_dataset = HeadlinesStocksDataset(val_encodings, df_val['label'].tolist())
# test_dataset = HeadlinesStocksDataset(test_encodings, df_test['label'].tolist())

train_dataset = CompanyHeadlinesStocksDataset(train_encodings, df_train['label'].tolist(), train_company_encodings, fin_train_encodings)
val_dataset = CompanyHeadlinesStocksDataset(val_encodings, df_val['label'].tolist(), val_company_encodings, fin_val_encodings)
test_dataset = CompanyHeadlinesStocksDataset(test_encodings, df_test['label'].tolist(), test_company_encodings, fin_test_encodings)

In [ ]:
BATCH_SIZE=16

train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
  )
val_data_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
  )
test_data_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
  )

In [ ]:
data_loaders = {'train': train_data_loader, 'validation': val_data_loader, 'test': test_data_loader}

# Training

In [ ]:
class CompModel(nn.Module):
  def __init__(self, base_model, comp_enc_size, fin_bert=None):
    super().__init__()

    self.base_model = base_model
    self.finbert = fin_bert
    self.classifier = nn.Sequential(nn.Linear(base_model.config.hidden_size + comp_enc_size + (3 if fin_bert is not None else 0), base_model.config.hidden_size),
                                    nn.Sigmoid(),
                                    nn.Linear(base_model.config.hidden_size, 1),
                                    nn.Sigmoid()
                                    )

  def forward(self, input_ids, attention_mask, comp_input_ids, fin_input_ids, fin_attention_mask):
    features = self.base_model(input_ids, attention_mask=attention_mask).pooler_output

    concat_features = torch.cat((features, comp_input_ids), dim = 1)

    if self.finbert is not None:
      fin_sent = self.finbert(fin_input_ids, attention_mask=fin_attention_mask).logits
      concat_features = torch.cat((concat_features, fin_sent), dim = 1)

    return self.classifier(concat_features)


In [ ]:
comp_model = CompModel(model, len(train_company_encodings['input_ids'][0])).to(device=device)

In [ ]:
EPOCHS = 5
optimizer = AdamW(comp_model.parameters(), lr=5e-5)

loss_fn = nn.BCELoss().to(device)

In [ ]:
history = train(comp_model, EPOCHS, train_data_loader, loss_fn, optimizer)

In [ ]:
metrics = comp_eval_model(comp_model, loss_fn, sets=['test'])

In [ ]:
confusion_matrix(metrics['test']['pred'], df_test['label'].tolist())

array([[218, 185],
       [168, 212]])

In [ ]:
metrics['test']['accuracy']

0.5491698595146871

In [ ]:
plot(history)

In [ ]:
model.save_pretrained("model")